# Test/Predict the saved Model..

In [1]:
import numpy as np
import cv2
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from pathlib import Path

2023-07-29 21:17:38.728762: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow.keras.backend as K

In [4]:
data = np.load("/home/sullah/PycharmProjects/EDSR/Alphorn_Project/for_professor_method_after_augmentation_256x256_another_try/combined_inputs.npy", mmap_mode='r+')

labels = np.load("/home/sullah/PycharmProjects/EDSR/Alphorn_Project/dataset_after_augmentation_256x256/del_data_after_augmentation_256x256.npy", mmap_mode='r+')

reference_images = np.load("/home/sullah/PycharmProjects/EDSR/Alphorn_Project/dataset_after_augmentation_256x256/healthy_data_after_augmentation_256x256.npy", mmap_mode='r+')

print(data.shape)
print(labels.shape)
print(reference_images.shape)

(1900, 32, 256, 256, 2)
(1900, 32, 256, 256, 1)
(1900, 32, 256, 256, 1)


In [5]:
test_data = data[1880:1900]
test_labels = labels[1880:1900]
test_reference_images = reference_images[1880:1900]

print(test_data.shape)
print(test_labels.shape)
print(test_reference_images.shape)

(20, 32, 256, 256, 2)
(20, 32, 256, 256, 1)
(20, 32, 256, 256, 1)


In [6]:
# def PSNR(y_true,y_pred):
#   mse=tf.reduce_mean( (y_true - y_pred) ** 2 )
#   return 20 * log10(1 / (mse ** 0.5))

# def log10(x):
#   numerator = tf.math.log(x)
#   denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
#   return numerator / denominator

# def pixel_MSE(y_true,y_pred):
#   return tf.reduce_mean( (y_true - y_pred) ** 2 )


def PSNR(orig, pred):
	# cast the target images to integer
	orig = orig * 255.0
	orig = tf.cast(orig, tf.uint8)
	orig = tf.clip_by_value(orig, 0, 255)
	# cast the predicted images to integer
	pred = pred * 255.0
	pred = tf.cast(pred, tf.uint8)
	pred = tf.clip_by_value(pred, 0, 255)
	# return the psnr
	return tf.image.psnr(orig, pred, max_val=255)

In [7]:
def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

In [8]:
# load the saved model
model_encoder = tf.keras.models.load_model('Model_2_encoder_mapping_GT_imgs_Latenet_spaces1.h5', compile = False)
model_decoder = tf.keras.models.load_model('ConvLSTM_with_Pooling_with_MSE_autoencoder_decoder-decoder_fully_ConvLSTM1.h5', compile = False)

2023-07-29 21:17:41.075259: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-29 21:17:41.076749: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-29 21:17:41.118051: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2023-07-29 21:17:41.118121: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-29 21:17:41.120720: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-07-29 21:17:41.120819: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [9]:
model_encoder.summary()
model_decoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256, 256, 2)] 0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 32, 256, 256, 32)  39296     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 32, 128, 128, 32)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 128, 128, 32)  128       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 32, 128, 128, 192) 1549056   
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 32, 64, 64, 192)   0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 64, 64, 192)   768 

In [10]:
x_test = test_data
print(x_test.shape)

(20, 32, 256, 256, 2)


In [11]:
prediction = model_decoder(model_encoder.predict(x_test, batch_size=1)) 
print(prediction.shape)
prediction = np.asarray(prediction)

2023-07-29 21:17:44.793178: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-07-29 21:17:44.811592: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
2023-07-29 21:17:45.414932: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:592] layout failed: Invalid argument: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/mul_2' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/add_5', 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/Tanh_1' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/mul_5', 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/convolution_7' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/add_6', 'encoder/conv_lst_m2d/while/body/_1/encoder/conv_lst_m2d

(20, 32, 256, 256, 1)


In [12]:
prediction = prediction.reshape((20*32, 256, 256, 1))
test_labels = test_labels.reshape((20*32, 256, 256, 1))
test_reference_images = test_reference_images.reshape((20*32, 256, 256, 1))

print(prediction.shape)
print(test_labels.shape)
print(test_reference_images.shape)

(640, 256, 256, 1)
(640, 256, 256, 1)
(640, 256, 256, 1)


In [13]:
frames = x_test.shape[0]

print(frames)
for i in range(640):
    case_num = 1881 + (i // 32)
    pred = prediction[i].astype('float32')            
    original = test_labels[i].astype('float32')
    ref = test_reference_images[i].astype('float32')
    
#     print('case #', case_num, 'frame #', (i + 1), 'PSNR', PSNR(original[i],pred[i]))
#     print('case #', case_num, 'frame #', (i + 1), 'pearson_r', pearson_r(original[i],pred[i]))
############################################################################################################
    plt.figure(figsize=(15 / 2.54, 5 / 2.54), dpi=600)
    plt.gca().set_axis_off()
    plt.axis('off')
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.1, hspace=0.0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
############################################################################################################          
#     SR_pred = SR_pred * 255.0

    # showing image
    ax1 = plt.subplot(1, 3, 1)
    ax1.imshow(ref) 
    ax1.set_title("Reference Image")
    ax1.axis('off')
    
    # showing image
    ax2 = plt.subplot(1, 3, 2)
    ax2.imshow(pred) 
    ax2.set_title("Predicted Image")
    ax2.axis('off')
    
    # showing image
    ax3 = plt.subplot(1, 3, 3)
    ax3.imshow(original)
    ax3.set_title("Actual Image")
    ax3.axis('off')
            
#   print('PSNR', PSNR(original[i],pred[i])) #can be like this too original[0][i]
#   print('pearson_r', pearson_r(original[0][i],pred[count][i]))
    
    path = '/home/sullah/PycharmProjects/EDSR/Alphorn_Project/numerical_results/%d_output/' % case_num
    Path(path).mkdir(parents=True, exist_ok=True)
    os.chdir(path)      
    plt.savefig('Saeed_SR_Pred_frame_%d_output_%d.png' % (i+12161, case_num))
    plt.close('all')

20
